In [1]:
import os

In [2]:
list_file = list(os.walk('./vehicle'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
len(list_file)

1401

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./vehicle/' + file, 'r', encoding='UTF-8') as f:
        list_row = json.loads(f.read())
        
        # = = = = = = = = = = = = = = =
        
        list_vehicle = []
        for row in list_row:
            make = row['fitment']['Vcdb']['MakeName'].strip()
            model = row['fitment']['Vcdb']['ModelName'].strip()
            if len(row['fitment']['Vcdb']['Engines']) == 0:
                engine = ''
            elif 'Cylinders' in row['fitment']['Vcdb']['Engines'][0]:
                engine = row['fitment']['Vcdb']['Engines'][0]['BlockType'].strip() + str(row['fitment']['Vcdb']['Engines'][0]['Cylinders']) + ' ' +  str(row['fitment']['Vcdb']['Engines'][0]['Liter']) + 'L'
            else:
                engine = str(row['fitment']['Vcdb']['Engines'][0]['Liter']) + 'L'
            year = int(row['fitment']['Vcdb']['Year'].strip())

            list_vehicle.append({'make': make, 'model': model, 'engine': engine, 'year': year})

        # = = = = = = = = = = = = = = =
        
        dict_vehicle_1 = {}
        for vehicle in list_vehicle:
            if vehicle['make'] not in dict_vehicle_1:
                dict_vehicle_1[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
            else:
                if vehicle['model'] not in dict_vehicle_1[vehicle['make']]:
                    dict_vehicle_1[vehicle['make']][vehicle['model']] = [vehicle['year']]
                else:
                    dict_vehicle_1[vehicle['make']][vehicle['model']].append(vehicle['year'])

        # = = = = = = = = = = = = = = =
        
        dict_vehicle_2 = {}
        for vehicle in list_vehicle:
            if vehicle['make'] not in dict_vehicle_2:
                dict_vehicle_2[vehicle['make']] = {vehicle['model']: {vehicle['engine']: [vehicle['year']]}}
            else:
                if vehicle['model'] not in dict_vehicle_2[vehicle['make']]:
                    dict_vehicle_2[vehicle['make']][vehicle['model']] = {vehicle['engine']: [vehicle['year']]}
                else:
                    if vehicle['engine'] not in dict_vehicle_2[vehicle['make']][vehicle['model']]:
                        dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']] = [vehicle['year']]
                    else:
                        dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']].append(vehicle['year'])
                        
        # = = = = = = = = = = = = = = =
        
        list_vehicle_1 = []
        for make in dict_vehicle_1:
            for model in dict_vehicle_1[make]:
                year_max = max(dict_vehicle_1[make][model])
                year_min = min(dict_vehicle_1[make][model])
                if year_max == year_min:
                    list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min))
                else:
                    list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

        list_vehicle_1.sort()

        vehicle_1 = '\n'.join(list_vehicle_1)
        
        # = = = = = = = = = = = = = = =
        
        list_vehicle_2 = []
        for make in dict_vehicle_2:
            for model in dict_vehicle_2[make]:
                for engine in dict_vehicle_2[make][model]:
                    year_max = max(dict_vehicle_2[make][model][engine])
                    year_min = min(dict_vehicle_2[make][model][engine])
                    if year_max == year_min:
                        list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + ' - ' + engine)
                    else:
                        list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' - ' + engine)

        list_vehicle_2.sort()

        vehicle_2 = '\n'.join(list_vehicle_2)
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame([{'No.': int(file.replace('.txt', '')),
                                 'Vehicle_1': vehicle_1,
                                 'Vehicle_2': vehicle_2}])
        
        # = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
df.to_excel('./vehicle.xlsx', index=False)

df

Progress: 100%|█████████████████████████| 1401/1401 [00:07<00:00, 181.13it/s]


,No.,Vehicle_1,Vehicle_2
0,1,Chevrolet Camaro 1993-1998\nPontiac Firebird 1...,Chevrolet Camaro 1993-1994 - V6 3.4L\nChevrole...
1,2,,
2,3,Cadillac CTS 2008-2015\nCadillac Escalade 2015...,Cadillac CTS 2008-2014 - V6 3.6L\nCadillac CTS...
3,4,Buick Enclave 2008-2017,Buick Enclave 2008-2017 - V6 3.6L
4,5,Chevrolet Malibu 2005-2007,Chevrolet Malibu 2005-2007 - L4 2.2L\nChevrole...
...,...,...,...
1396,1397,Honda Passport 1998-2002\nIsuzu Rodeo 1998-2004,Honda Passport 1998-2002 - V6 3.2L\nIsuzu Rode...
1397,1398,Volvo V70 2005-2007\nVolvo XC70 2005-2008,Volvo V70 2005-2007 - L5 2.4L\nVolvo V70 2005-...
1398,1399,Volvo XC90 2003-2014,Volvo XC90 2003-2006 - L6 2.9L\nVolvo XC90 200...
1399,1400,Volvo XC90 2004-2014,Volvo XC90 2004-2006 - L6 2.9L\nVolvo XC90 200...
